In [23]:
path = '/home/vscode/workspace/traces/session_main6/ust/uid/1000/64-bit/'

In [24]:
import sys
# Add paths to tracetools_analysis and tracetools_read.
# There are two options:
#   1. from source, assuming a workspace with:
#       src/tracetools_analysis/
#       src/ros2/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../ros2/ros2_tracing/tracetools_read/')
#   2. from Debian packages, setting the right ROS 2 distro:
#ROS_DISTRO = 'rolling'
#sys.path.insert(0, f'/opt/ros/{ROS_DISTRO}/lib/python3.8/site-packages')
import datetime as dt

from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [28]:
events = load_file(path)

found converted file: /home/vscode/workspace/traces/session_main7/ust/uid/1000/64-bit/converted


In [29]:
handler = Ros2Handler().process(events)

RequiredEventNotFoundError: missing events: {'Ros2Handler': {'ros2:rcl_init'}}

In [ ]:
data_util = Ros2DataModelUtil(handler.data)

In [ ]:
data_util.get_callback_symbols()

{101552287235568: 'TestNode::TestNode()::{lambda()#1}',
 101552287036696: 'rclcpp::TimeSource::NodeState::attachNode(std::shared_ptr<rclcpp::node_interfaces::NodeBaseInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeTopicsInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeGraphInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeServicesInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeLoggingInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeClockInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeParametersInterface>)::{lambda(std::shared_ptr<rcl_interfaces::msg::ParameterEventconst>)#1}',
 101552287274464: 'TestNode::TestNode()::{lambda()#2}'}

In [ ]:
output_notebook()
psize = 450
colours = ['#29788E', '#DD4968', '#410967']

Loading BokehJS ...

In [ ]:
callback_symbols = data_util.get_callback_symbols()

In [ ]:
callback_symbols

{101552287235568: 'TestNode::TestNode()::{lambda()#1}',
 101552287036696: 'rclcpp::TimeSource::NodeState::attachNode(std::shared_ptr<rclcpp::node_interfaces::NodeBaseInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeTopicsInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeGraphInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeServicesInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeLoggingInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeClockInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeParametersInterface>)::{lambda(std::shared_ptr<rcl_interfaces::msg::ParameterEventconst>)#1}',
 101552287274464: 'TestNode::TestNode()::{lambda()#2}'}

In [ ]:
# Plot durations in one plot
earliest_date = None
for obj, symbol in callback_symbols.items():
    duration_df = data_util.get_callback_durations(obj)
    thedate = duration_df.loc[:, 'timestamp'].iloc[0]
    if earliest_date is None or thedate <= earliest_date:
        earliest_date = thedate

starttime = earliest_date.strftime('%Y-%m-%d %H:%M')
duration = figure(
    title='Callback durations',
    x_axis_label=f'start ({starttime})',
    y_axis_label='duration (ms)',
    width=psize, height=psize,
)

colour_i = 0
for obj, symbol in callback_symbols.items():
    # Filter out internal subscriptions
    owner_info = data_util.get_callback_owner_info(obj)
    if not owner_info or '/parameter_events' in owner_info:
        continue

    duration_df = data_util.get_callback_durations(obj)
    source = ColumnDataSource(duration_df)
    duration.title.align = 'center'
    duration.line(
        x='timestamp',
        y='duration',
        legend_label=str(symbol),
        line_width=2,
        source=source,
        line_color=colours[colour_i],
    )
    colour_i += 1
    duration.legend.label_text_font_size = '11px'
    duration.xaxis[0].formatter = DatetimeTickFormatter(seconds='%Ss')

show(duration)

In [ ]:
from numpy import timedelta64


test_df = data_util.get_callback_durations([*callback_symbols][0])
test_df['delta'] = test_df['timestamp'].diff()
test_df['delta'] = test_df['delta'].fillna(timedelta64(0))
test_df['delta'] = test_df['delta'].dt.total_seconds() * 1000 # convert to ms
test_df

,timestamp,duration,delta
0,2024-03-24 09:14:16.254551921,0 days 00:00:00.000108870,0.00000
2,2024-03-24 09:14:16.254716431,0 days 00:00:00.000014610,0.16451
4,2024-03-24 09:14:16.254766771,0 days 00:00:00.000013620,0.05034
6,2024-03-24 09:14:16.254815051,0 days 00:00:00.000013160,0.04828
8,2024-03-24 09:14:16.254862521,0 days 00:00:00.000013060,0.04747
...,...,...,...
58125,2024-03-24 09:14:25.579921061,0 days 00:00:00.000037050,0.19264
58126,2024-03-24 09:14:25.580111961,0 days 00:00:00.000038370,0.19090
58127,2024-03-24 09:14:25.580293321,0 days 00:00:00.000036770,0.18136
58128,2024-03-24 09:14:25.580475791,0 days 00:00:00.000036080,0.18247


In [ ]:
test_df.describe()

,timestamp,duration,delta
count,58045,58045,58045.000000
mean,2024-03-24 09:14:20.973050112,0 days 00:00:00.000035331,0.160670
min,2024-03-24 09:14:16.254551921,0 days 00:00:00.000012860,0.000000
25%,2024-03-24 09:14:18.661300992,0 days 00:00:00.000031310,0.133810
50%,2024-03-24 09:14:20.997056512,0 days 00:00:00.000036550,0.180730
75%,2024-03-24 09:14:23.270746112,0 days 00:00:00.000038470,0.187990
max,2024-03-24 09:14:25.580645061,0 days 00:00:00.000280200,0.440289
std,NaN,0 days 00:00:00.000009357,0.044093


In [ ]:
# plot difference between starts of callbacks

earliest_date = None
for obj, symbol in callback_symbols.items():
    duration_df = data_util.get_callback_durations(obj)
    thedate = duration_df.loc[:, 'timestamp'].iloc[0]
    if earliest_date is None or thedate <= earliest_date:
        earliest_date = thedate

starttime = earliest_date.strftime('%Y-%m-%d %H:%M')

duration = figure(
    title='Callback start delta',
    x_axis_label=f'start ({starttime})',
    y_axis_label='time since start (ms)',
    width=psize, height=psize,
)

colour_i = 0
for obj, symbol in [*callback_symbols.items()][:1]:
    # Filter out internal subscriptions
    owner_info = data_util.get_callback_owner_info(obj)
    if not owner_info or '/parameter_events' in owner_info:
        continue

    duration_df = data_util.get_callback_durations(obj)

    # sort by timestamp
    duration_df = duration_df.sort_values(by='timestamp')

    # calculate time since previous start
    duration_df['delta'] = duration_df['timestamp'].diff()
    duration_df['delta'] = duration_df['delta'].fillna(timedelta64(0))
    duration_df['delta'] = duration_df['delta'].dt.total_seconds() * 1000 # convert to ms

    source = ColumnDataSource(duration_df)
    duration.title.align = 'center'
    duration.line(
        x='timestamp',
        y='delta',
        legend_label=str(symbol),
        line_width=2,
        source=source,
        line_color=colours[colour_i],
    )
    colour_i += 1
    duration.legend.label_text_font_size = '11px'
    duration.xaxis[0].formatter = DatetimeTickFormatter(seconds='%Ss')

show(duration)
    